# Import Libraries

In [ ]:
import argparse
import json
import math
import os
import pickle
import random
import time
from collections import defaultdict
from datetime import datetime
from itertools import chain, combinations

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.optimize
from scipy.stats import f as fdist, ttest_ind, weibull_min
from sklearn.linear_model import LinearRegression, PoissonRegressor
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad

Set Seeds for reproducibility

In [ ]:
seed = 138
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

In [ ]:
def pretty(vector):
    # Convert to tensor if not already
    if not isinstance(vector, torch.Tensor):
        vector = torch.tensor(vector)
    vlist = vector.flatten().tolist()  # Using flatten() instead of view(-1)
    return "[" + ", ".join("{:+.4f}".format(vi) for vi in vlist) + "]"

In [ ]:
def weighted_poisson_nll(input, target, weights):
    # Standard Poisson NLL terms
    loss = input - target * torch.log(input + 1e-8)
    # Multiply each observation's loss by its frequency weight
    weighted_loss = (loss * weights).mean()
    return weighted_loss

# Domain Generalisation Algorithms

### Poisson Deviance Loss Implementation

In this cell, we define the `PoissonDevianceLoss` class, a custom loss function implemented using PyTorch. This loss function is specifically designed for Poisson regression tasks, where the observed target values are modeled as Poisson-distributed.

#### Key Features:
- **Purpose**: Measures the deviance between observed values (`y_true`) and predicted mean values (`y_pred`).
- **Implementation Details**:
  - Ensures that predicted values are strictly positive (required for Poisson regression).
  - Handles edge cases such as `0 * log(0)` safely.
  - Implements the standard deviance formula for Poisson regression.

#### Parameters:
- `y_true`: Observed target values (must be a PyTorch tensor).
- `y_pred`: Predicted mean values (must be positive and a PyTorch tensor).

#### Returns:
- The computed deviance loss as a PyTorch tensor.

This function can be used as a loss function in training models for Poisson regression tasks.


In [ ]:
class PoissonDevianceLoss(nn.Module):
    def __init__(self):
        super(PoissonDevianceLoss, self).__init__()

    def forward(self, y_true, y_pred):
        """
        Compute the deviance loss for Poisson regression.
        """
        if torch.any(y_pred <= 0):
            raise ValueError("Predicted values must be strictly positive.")

        # Compute deviance components
        term1 = y_true * torch.log(y_true / y_pred + 1e-8)  # Add small epsilon to avoid log(0)
        term1 = torch.where(y_true > 0, term1, torch.zeros_like(term1))  # Handle 0*log(0) -> 0
        term2 = y_true - y_pred

        # Deviance formula
        deviance = 2 * torch.sum(term1 - term2)
        return deviance

### Invariant Risk Minimization (IRM) Class Implementation

In this cell, we define the `InvariantRiskMinimization` class, which implements the IRM algorithm for domain generalization. The objective of IRM is to learn invariant predictors across different environments, minimizing risk while ensuring the invariance of the learned model.

#### Key Components:
1. **Initialization**:
   - Converts the inputs (`exog`, `endog`, and `freqs`) to PyTorch tensors for compatibility.
   - Performs hyperparameter tuning by training the model with different regularization values (`reg`), selecting the best model based on validation error.

2. **Train Method**:
   - Trains the IRM model using a custom loss function incorporating a weighted Poisson Negative Log-Likelihood loss.
   - Computes a penalty to ensure the invariance of the gradients across environments.

3. **Solution**:
   - Provides the learned model parameters as a NumPy array.

4. **Prediction**:
   - Uses the trained model to predict Poisson rates (`lambda`) for a given input.

#### Parameters:
- `exog`: Exogenous variables (input features for environments).
- `endog`: Endogenous variables (target values for environments).
- `freqs`: Weights or frequencies associated with each environment.
- `args`: Dictionary of hyperparameters, including learning rate and number of iterations.

#### Notable Features:
- Includes gradient-based regularization to enforce invariance.
- Uses PyTorch for tensor operations, loss computation, and optimization.
- Supports weighted Poisson regression for handling environment-specific weights.

This class serves as a foundation for training robust predictive models that generalize well across varying environments.


In [ ]:
class InvariantRiskMinimization(object):
    def __init__(self, exog, endog, freqs, args):
        """
        Initialize and train the model with the given environments and arguments.

        Args:
            environments: List of tuples (x, y), where x and y are tensors for each environment.
            args: Dictionary of hyperparameters, e.g., learning rate and number of iterations.
        """
        self.best_reg = 0
        self.best_err = float("inf")

        exog = torch.tensor(np.array(exog), dtype=torch.float32)
        endog = torch.tensor(np.array(endog), dtype=torch.float32)
        freqs = torch.tensor(np.array(freqs), dtype=torch.float32)

        x_val = exog[-1]
        y_val = endog[-1]
        for reg in [0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]:
            self.train(exog[:-1], endog[:-1], freqs[:-1], args, reg=reg)
            err = torch.nn.PoissonNLLLoss(log_input=False, reduction='mean')(torch.exp(x_val @ self.solution()), y_val)
            #err = torch.mean((torch.exp(x_val @ self.solution()) - y_val) ** 2).item()

            if args.verbose:
                print(f"IRM (reg={reg:.3f}) has validation error: {err:.3f}.")

            if err < self.best_err:
                self.best_err = err
                self.best_reg = reg
                self.best_phi = self.phi.clone()

        #self.phi = self.best_phi

    def train(self, exog, endog, freqs, args, reg=0):
        """
        Train the IRM model using the given environments.

        Args:
            environments: List of tuples (x, y), where x and y are tensors for each environment.
            args: Dictionary of hyperparameters.
            reg: Regularization coefficient for IRM penalty.
        """
        dim_x = exog[0].size(1)
        self.phi = nn.Parameter(torch.empty(dim_x, dim_x))
        nn.init.xavier_uniform_(self.phi)
        self.w = nn.Parameter(torch.empty(dim_x, 1))
        nn.init.xavier_uniform_(self.w)

        optimizer = torch.optim.Adam([self.phi, self.w], lr=args.lr, weight_decay=1e-5)

        # Custom weighted Poisson loss function

        for iteration in range(args.n_iterations):
            total_penalty = 0
            total_error = 0

            for x_e, y_e, f in zip(exog, endog, freqs):
                input = torch.exp(x_e @ self.phi @ self.w)
                # Apply weighted loss for this environment
                error_e = weighted_poisson_nll(input, y_e, f)
                #error_e = nn.PoissonNLLLoss(log_input=False, reduction='mean')(input, y_e)

                # Compute gradient and ensure it's scalar
                grad_e = grad(error_e, self.w, create_graph=True)[0]
                penalty_e = grad_e.pow(2).mean()

                total_penalty += penalty_e
                total_error += error_e

            loss = reg * total_error + (1 - reg) * total_penalty

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if args.verbose and iteration % 100 == 0:
                w_str = pretty(self.solution())
                print(
                    "{:05d} | {:.5f} | {:.5f} | {:.5f} | {}".format(
                        iteration, reg, total_error.item(), total_penalty.item(), w_str
                    ))

    def solution(self):
        """
        Returns the learned model parameters as a numpy array.
        """
        return (self.phi @ self.w).view(-1, 1).detach().numpy()

    def predict(self, X):

        if isinstance(X, np.ndarray):
            X = torch.tensor(X, dtype=torch.float32)

        # Compute the logits using the learned parameters
        pred = X @ torch.tensor(self.solution(), dtype=torch.float32)
        rates = torch.exp(pred)

        # Return the predicted Poisson rates (lambda)
        return rates.detach().numpy()

### Maximum Mean Discrepancy (MMD) Class Implementation

This cell introduces the `MaximumMeanDiscrepancy` class, a domain generalization method that minimizes the distributional discrepancy between environments. The MMD algorithm uses a kernel-based measure to ensure that learned representations are invariant across environments.

#### Key Components:
1. **Initialization**:
   - Converts input features (`exog`), targets (`endog`), and weights (`freqs`) to PyTorch tensors.
   - Iteratively trains the model for various values of the regularization parameter (`gamma`) to select the best-performing model based on validation error.

2. **Train Method**:
   - Trains the MMD model by minimizing a weighted Poisson negative log-likelihood loss and the MMD penalty.
   - Computes pairwise discrepancies between environments using the specified kernel type (e.g., Gaussian, mean covariance).

3. **Solution**:
   - Provides the learned model parameters in a usable format (NumPy array).

4. **MMD Calculation**:
   - Includes two types of kernel computations:
     - **Gaussian Kernel**: Uses multiple bandwidths for robust similarity computation.
     - **Mean-Covariance Kernel**: Considers both mean and covariance differences between distributions.

5. **Prediction**:
   - Uses the learned parameters to predict Poisson rates (`lambda`) for a given input.

#### Parameters:
- `exog`: Exogenous variables (input features).
- `endog`: Endogenous variables (target values).
- `freqs`: Weights associated with each environment.
- `args`: Dictionary of hyperparameters including learning rate, number of iterations, and kernel type.

#### Notable Features:
- Supports various kernel types for measuring discrepancies between environments.
- Implements a custom pairwise distance computation (`my_cdist`) for Gaussian kernels.
- Integrates regularization to balance predictive accuracy and distributional alignment.

This class enables robust training of models that generalize well across different environments by explicitly minimizing inter-environment discrepancies.


In [ ]:
class MaximumMeanDiscrepancy(object):
    def __init__(self, exog, endog, freqs, args):
        best_reg = 0
        best_err = 1e6

        exog = torch.tensor(np.array(exog), dtype=torch.float32)
        endog = torch.tensor(np.array(endog), dtype=torch.float32)
        freqs = torch.tensor(np.array(freqs), dtype=torch.float32)

        x_val = exog[-1]
        y_val = endog[-1]

        for gamma in [1e-3, 1e-2, 1e-1, 1]:
            self.train(exog[:-1], endog[:-1], freqs[:-1], args, gamma=gamma)
            err = torch.mean((torch.exp(x_val @ self.solution()) - y_val) ** 2).item()

            if args.verbose:
                print("MMD (gamma={:.3f}) has {:.3f} validation error.".format(gamma, err))

            if err < best_err:
                best_err = err
                best_gamma = gamma
                best_phi = self.phi.clone()
        # Save the best model parameters
        #self.phi = best_phi

    def train(self, exog, endog, freqs, args, gamma=0):
        dim_x = exog[0].size(1)
        num_envs = exog.shape[0]

        # Initialize phi and weights
        self.phi = nn.Parameter(torch.empty(dim_x, dim_x))
        nn.init.xavier_uniform_(self.phi)
        self.w = torch.empty(dim_x, 1)
        nn.init.xavier_uniform_(self.w)
        self.w.requires_grad = True

        # Optimizer and loss function for Poisson regression
        opt = torch.optim.Adam([self.phi, self.w], lr=args.lr, weight_decay=1e-5)
        #poisson_loss = torch.nn.PoissonNLLLoss(log_input=False, reduction='mean')

        for iteration in range(args.n_iterations):
            penalty = 0
            error = 0

            # Compute error and MMD penalty for each pair of environments
            for i, (x_e1, y_e1, f) in enumerate(zip(exog, endog, freqs)):
                input = torch.exp(x_e1 @ self.phi @ self.w)
                error += weighted_poisson_nll(input, y_e1, f) # * f

                for j, (x_e2, y_e2, f) in enumerate(zip(exog, endog, freqs)):
                    if i < j:
                        penalty += self.mmd(x_e1 @ self.phi, x_e2 @ self.phi, args.kernel_type) #* f

            # Normalize penalty
            error /= num_envs
            if num_envs > 1:
                penalty /= (num_envs * (num_envs - 1) / 2)

            # Optimize the combined loss
            opt.zero_grad()
            loss = error + gamma * penalty
            loss.backward()
            opt.step()

            if args.verbose and iteration % 100 == 0:
                w_str = pretty(self.solution())
                print(
                    "{:05d} | {:.5f} | {:.3f} | {:.3f} | {}".format(
                        iteration, gamma, error, penalty, w_str
                    ))

    def solution(self):
        """Returns the learned model parameters as a numpy array."""
        return (self.phi @ self.w).view(-1, 1).detach().numpy()

    def mmd(self, x, y, kernel_type):
        if kernel_type == "gaussian":
            Kxx = self.gaussian_kernel(x, x).mean()
            Kyy = self.gaussian_kernel(y, y).mean()
            Kxy = self.gaussian_kernel(x, y).mean()
            return Kxx + Kyy - 2 * Kxy

        elif kernel_type == "mean_cov":
            mean_x = x.mean(0, keepdim=True)
            mean_y = y.mean(0, keepdim=True)
            cent_x = x - mean_x
            cent_y = y - mean_y
            cova_x = (cent_x.t() @ cent_x) / (len(x) - 1)
            cova_y = (cent_y.t() @ cent_y) / (len(y) - 1)

            mean_diff = (mean_x - mean_y).pow(2).mean()
            cova_diff = (cova_x - cova_y).pow(2).mean()

            return mean_diff + cova_diff
        else:
            raise ValueError("Unknown kernel type")

    def gaussian_kernel(self, x, y, gamma=[0.001, 0.01, 0.1, 1, 10, 100]):
        D = self.my_cdist(x, y)
        K = torch.zeros_like(D)
        for g in gamma:
            K.add_(torch.exp(D.mul(-g)))
        return K

    def my_cdist(self, x1, x2):
        x1_norm = x1.pow(2).sum(dim=-1, keepdim=True)
        x2_norm = x2.pow(2).sum(dim=-1, keepdim=True)
        return torch.addmm(
            x2_norm.transpose(-2, -1), x1, x2.transpose(-2, -1), alpha=-2
        ).add_(x1_norm).clamp_min_(1e-30)

    def predict(self, X):

        if isinstance(X, np.ndarray):
            X = torch.tensor(X, dtype=torch.float32)

        sol = torch.tensor(self.solution(), dtype=torch.float32)   # Flatten the solution
        pred = X @ sol
        rates = torch.exp(pred)
        return rates.detach().numpy()  # Apply exponential for Poisson

### Empirical Risk Minimizer (ERM) Class Implementation

This cell implements the `EmpiricalRiskMinimizer` class, which represents a baseline approach for domain generalization. ERM aggregates all data across environments to train a single model, ignoring potential differences between environments.

#### Key Components:
1. **Initialization**:
   - Combines input features (`exog`), targets (`endog`), and weights (`freqs`) from all environments.
   - Trains a Poisson regression model using scikit-learn's `PoissonRegressor` with sample weighting to account for varying frequencies across environments.

2. **Model Training**:
   - The regression model learns a single set of parameters (`w`) across all environments by minimizing the empirical risk (negative log-likelihood for Poisson regression).

3. **Prediction**:
   - Provides predictions for new input data (`X`) using the trained regression model.

4. **Solution**:
   - Exposes the learned model parameters (`w`) for further analysis or interpretation.

#### Parameters:
- `exog`: List of exogenous variables (covariates) for each environment.
- `endog`: List of endogenous variables (target values) for each environment.
- `freqs`: List of weights or frequencies for each environment.
- `args`: Hyperparameters, including the number of iterations for model training.

#### Use Case:
ERM is a straightforward approach for modeling data when the goal is to optimize predictive performance on an aggregated dataset, without explicitly considering domain-specific characteristics.

This implementation highlights the importance of baseline methods in evaluating advanced domain generalization algorithms.


In [ ]:
class EmpiricalRiskMinimizer(object):
    def __init__(self, exog, endog, freqs, args):
        # x is the covariates matrix
        # y is the depedent variable
        x_all = np.concatenate([e for e in exog])
        y_all = np.concatenate([e for e in endog])
        freq_all = np.concatenate([e for e in freqs])

        # w = LinearRegression(fit_intercept=False).fit(x_all, y_all).coef_
        self.model = PoissonRegressor(max_iter=args.n_iterations, verbose=0)
        self.model.fit(x_all, y_all.ravel(), sample_weight=freq_all.ravel())
        w = self.model.coef_
        self.w = w.reshape(-1, 1)

    def predict(self, X):
        res = self.model.predict(X)
        return res

    def solution(self):
        return self.w

# Synthetic Data Generation

Define input parameters

In [ ]:
verbose = 1
lr = 0.1
n_iterations = 1000
method = "IRM"
covariate_dim = 10
true_mu = 0.8
emiter = 100
kernel_type = "gaussian"
start_alpha = 6.0
start_beta = 4.0
start_mu = 0.6

### Synthetic Data Generator with Hawkes Process

The class creates multiple environments with causal and acausal covariates and simulates events using a Hawkes process. Here's an overview:

- **Causal and Acausal Covariates**: Covariates are generated for each environment, with causal relationships modeled explicitly.
- **Hawkes Process**: Events are simulated using a discrete-time Hawkes process with a Weibull decay kernel, capturing time-dependent dependencies.
- **Environment Simulation**: Multiple environments are constructed with varying noise levels, and event counts are generated for each.

The `SyntheticEnvs` class provides methods to:
1. Generate causal and acausal covariates.
2. Simulate events using a Hawkes process.
3. Compute the "true" underlying solution for the synthetic dataset.


In [ ]:
class SyntheticEnvs:
    def __init__(self, n_cty, n_day, alpha, beta, mu, env_list, dim, h):
        """
        Initializes variables for synthetic data generation
        assume n_cty is number of samples
        Days ,alpha, beta, mu, environment values, covariate dimension
        h is the seed for the random number generator
        covariates, weights generated here
        Only one layer of dimensionality, mu and cov constant across days
        """
        # Set seeds for all possible sources of randomness
        self.h = h
        np.random.seed(self.h)
        random.seed(self.h)
        os.environ['PYTHONHASHSEED'] = str(self.h)

        self.n_cty = n_cty
        self.n_day = n_day
        self.alpha = alpha
        self.beta = beta
        self.mu = mu

        # generate events w mob demo data for 1 env
        self.dim_x = dim // 2
        self.causal_cov = np.random.rand(self.n_cty, self.dim_x)
        if self.dim_x == 4:
            self.wxy = np.array([
                [1, -2, 3, 0],
                [-0.34, 2, 0, 0.5],
                [0, -2, -1, 0],
                [0, 1, -2.5, -2]
            ])
        else:
            self.wxy = np.random.rand(self.dim_x, self.dim_x) * 0.15 # scaled to [0, 0.25] range

        # generate acausal data and events for all envs
        covariate_groups, case_count_groups, lambda_groups = self.makeEnvs(env_list)

        self.case_count_envs = case_count_groups
        self.covariate_envs = covariate_groups
        self.lambda_envs = lambda_groups

    def hawkes_discrete_simulation(self, mu, R):
        """
        Simulates a Hawkes process with discrete time steps using the thinning method.
        generates exponentially decaying continuous time steps, then calculates probability
        of acceptance using discretized binned timestep

        Lambda = mu + Sum{R(X0)w(t-t-j) : t_j<t}
        Parameters:
            mu (numpy array): Background rate
            R (numpy array): Reproductive rate (shape: n_cty, 1).
            xw (numpy array): Covariates

        Returns:
            events (numpy array): event counts for each county (shape: n_cty, T).
            lambda_t (numpy array): conditional intensity values (shape: n_cty, T).
        """
        T = self.n_day  # time steps

        # Estimate lambda_max using Weibull maximum
        t_peak = self.alpha * ((self.beta - 1) / self.beta) ** (1 / self.beta) if self.beta > 1 else 0
        w_peak = weibull_min.pdf(t_peak, self.beta, scale=self.alpha)
        R_max = np.max(R)
        lambda_max = np.max(mu) + np.sum(R_max * w_peak)

        # outputs
        events = np.zeros((self.n_cty, T), dtype=int)
        lambda_t = np.zeros((self.n_cty, T))

        # samples events n_cty times for n_cty event rates
        for i in range(self.n_cty):
            t = 0
            # Add safety counter to prevent infinite loops
            iteration_count = 0
            max_iterations = 1000000  # Adjust this number as needed

            while t < T:
                # Add safety check
                iteration_count += 1
                if iteration_count > max_iterations:
                    print(f"Warning: Maximum iterations reached for county {i}")
                    break

                delta_t = np.random.exponential(1 / lambda_max)
                t_candidate = t + delta_t

                # Add minimum time step to ensure progress
                if delta_t < 1e-10:  # Prevent extremely small time steps
                    delta_t = 1e-10
                    t_candidate = t + delta_t

                if t_candidate >= T:
                    break

                t_discrete = int(np.floor(t_candidate))

                # triggering kernel influence
                past_events = np.where(events[i, :t_discrete] > 0)[0]
                hist_influence = np.sum([
                    R[i] * weibull_min.pdf(t_discrete - t_j, self.alpha, loc=0, scale=self.beta)
                    for t_j in past_events
                ])
                lambda_t_candidate = mu[i] + hist_influence

                if np.random.uniform(0, 1) <= (lambda_t_candidate / lambda_max):
                    events[i, t_discrete] += 1

                t = t_candidate

            # Compute lambda_t for all time steps
            for t in range(T):
                past_events = np.where(events[i, :t] > 0)[0]
                hist_influence = np.sum([
                    R[i] * weibull_min.pdf(t - t_j, self.beta, scale=self.alpha)
                    for t_j in past_events
                ])
                lambda_t[i, t] = mu[i] + hist_influence
            print(f"Sampling complete for county {i}")

        return events, lambda_t

    def trueSolution(self):
        w_reduced = np.sum(self.wxy, axis=1)
        sol = np.concatenate([w_reduced, np.zeros(self.dim_x)])
        # shape of sol is (2dim_x, 1); dim_x is number of causal covariates
        return sol

    def makeEnvs(self, env_list):
        """
        Makes a set of environments using an input env_list
        one set of acausal cov geenrated with env dependent noise for each env
        X: causal and acausal cov concatenated
        Y: y summed
        (X, Y) returned
        after which R will be poisson sampled from Y
        and case count will be generated from R, mu(same for all envs)
        init shuld return sets of case count and X for each env (observed variables for EM)
        """
        x = self.causal_cov # shape (n_cty, dim_x)
        y = x @ self.wxy
        wyz = np.random.rand(self.dim_x, self.dim_x)
        all_covariates = []
        all_lambdas = []
        for _, e in enumerate(env_list):
            z_e = y @ wyz + np.random.rand(self.n_cty, self.dim_x) * e
            cov = np.concatenate([x, z_e], axis=1)  # Concatenate vectors
            all_covariates.append(cov)

        w = np.sum(self.wxy, axis=1, keepdims=True)
        R = np.exp(x @ w).flatten() # shape (n_cty, 1)

        all_case_counts = []
        for e in range(len(env_list)):
            # generates event sequence for each environment
            print(f"Starting simulation for env {e}")
            case_count_e, lambda_e = self.hawkes_discrete_simulation(self.mu, R)
            all_case_counts.append(case_count_e)
            all_lambdas.append(lambda_e)

        return all_covariates, all_case_counts, all_lambdas

Synthetic data parameters

In [ ]:
n_day = 20
true_alpha = 1.0
true_beta = 1.81
env_list = [1.2, 1.8, 3, 0.2, 5]
covariate_dim = 10
true_mu = 0.8
n_cty = 20
seed = 138

In [ ]:
mu = np.repeat(true_mu, n_cty)
True_Data = SyntheticEnvs(
    n_cty,
    n_day,
    true_alpha,
    true_beta,
    mu,
    env_list,
    covariate_dim,
    seed
)

In [ ]:
data = {
    'case_count_all': True_Data.case_count_envs,
    'covariates_all': True_Data.covariate_envs,
    'true_weights': True_Data.trueSolution(),
    'true_lambdas': True_Data.lambda_envs,
    'true_params': {
        'true_alpha': true_alpha,
        'true_beta': true_beta,
        'true_mu': mu,
        'seed': seed,
        'env_list': env_list,
        'n_cty': n_cty,
        'n_day': n_day
    }
}

Visualise synthetic data

In [ ]:
# Plot 1: Lambda heatmaps
n_envs = len(data['true_lambdas'])
n_cols = min(3, n_envs)
n_rows = (n_envs + 2) // 3

fig1, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows))
if n_envs > 1:
    axes = axes.flatten()
else:
    axes = [axes]

for i, lambda_env in enumerate(data['true_lambdas']):
    sns.heatmap(lambda_env, ax=axes[i], cmap='YlOrRd')
    axes[i].set_title(f'Environment {i + 1}')
    axes[i].set_xlabel('Time')
    axes[i].set_ylabel('Location')

# Add parameter values as text at the bottom of the figure
param_text = f'μ = {true_mu:.2f}, α = {true_alpha:.2f}, β = {true_beta:.2f}'
fig1.text(0.5, 0.02, param_text, ha='center', fontsize=10)

# Adjust layout to make room for the parameter text
plt.tight_layout(rect=[0, 0.05, 1, 1])

for j in range(i + 1, len(axes)):
    axes[j].remove()

plt.show()

# Plot 2: Cumulative case counts
fig2, ax = plt.subplots(figsize=(10, 6))

colors = plt.cm.rainbow(np.linspace(0, 1, n_envs))
for i, cases in enumerate(data['case_count_all']):
    cumulative_cases = np.cumsum(cases.sum(axis=0))
    ax.plot(cumulative_cases, color=colors[i],
            label=f'Environment {i + 1}', linewidth=2)

ax.set_xlabel('Time')
ax.set_ylabel('Cumulative Cases')
ax.set_title('Cumulative Case Counts Across Environments')
ax.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Plot 3: Daily new cases
fig3, ax = plt.subplots(figsize=(10, 6))

for i, cases in enumerate(data['case_count_all']):
    daily_cases = cases.sum(axis=0)  # Sum across all locations for each day
    ax.plot(daily_cases, color=colors[i],
            label=f'Environment {i + 1}', linewidth=2)

ax.set_xlabel('Time')
ax.set_ylabel('Daily New Cases')
ax.set_title('Daily New Cases Across Environments')
ax.legend()
plt.grid(True, alpha=0.3)
plt.show()


Load Data

In [ ]:
case_count_all = data['case_count_all']
covariates_all = data['covariates_all']
true_weights = data['true_weights']
true_lambdas = data['true_lambdas']

# Preprocessing

True Parameters

In [ ]:
args.true_alpha = data['true_params']['true_alpha']
args.true_beta = data['true_params']['true_beta']
args.n_day = data['true_params']['n_day']
args.n_cty = data['true_params']['n_cty']
args.env_list = data['true_params']['env_list']
mu = data['true_params']['true_mu']

Initial guesses for Hawkes process

In [ ]:
env_data = [
    {
        "R0": np.exp(covariates_all[e] @ np.random.rand(args.covariate_dim)).reshape(-1, 1),
        "mus": np.full((n_cty, 1), args.start_mu),
        "lambda_t": np.zeros((n_cty, n_day)),
        "p_c_ij": None,
        "p_c_ii": None,
        "Q": None,
        "alpha": args.start_alpha,
        "beta": args.start_beta
    } for e in range(n_env)
]

tracking updates across iterations

In [ ]:
mus_prev, theta_prev = [None] * n_env, [None] * n_env
alphas_prev = [None] * n_env
betas_prev = [None] * n_env
alpha_delta = [[] for _ in range(n_env)]
beta_delta = [[] for _ in range(n_env)]
mus_delta = [[] for _ in range(n_env)]
theta_delta = [[] for _ in range(n_env)]

combined evironemnts for standardization

In [ ]:
covariates_full = np.vstack([covariates_all[e] for e in range(n_env)])
cov_mean = np.mean(covariates_full, axis=0)
cov_std = np.std(covariates_full, axis=0)
# Add small constant to avoid division by zero
cov_std = np.where(cov_std < 1e-10, 1e-10, cov_std)
# Standardize each environment using the global parameters
for e in range(n_env):
    covariates_all[e] = (covariates_all[e] - cov_mean) / cov_std

# E M Algorithm

In [ ]:
for itr in range(emitr):
    start_time = time.time()
    print("----------------------------------------------------------")
    print(f"Starting EM Iteration: {itr+1}")

    for e in range(n_env):
        # E-step
        T = (case_count_all[e].shape[1])
        R0_ext_j = np.repeat(env_data[e]["R0"], T, axis=0) # shape: n_cty*n_day, 1
        trig_comp = R0_ext_j * wblval(T, n_cty, env_data[e]["alpha"], 0, env_data[e]["beta"]) * (np.repeat(case_count_all[e], T, axis=0) > 0)
        mu_comp = np.tile(np.eye(T), (n_cty, 1)) * np.repeat(env_data[e]["mus"], T, axis=0)
        lambda_t = np.sum(mu_comp + trig_comp, axis=1, keepdims=True)
        env_data[e]["lambda_t"] = lambda_t
        env_data[e]["p_c_ij"] = np.divide(trig_comp, lambda_t, where= lambda_t != 0)
        env_data[e]["p_c_ii"] = np.divide(mu_comp, lambda_t, where= lambda_t != 0)

        P_c_j = env_data[e]["p_c_ij"].reshape(n_day, n_day*n_cty)
        P_c_j = np.reshape(np.sum(P_c_j, axis=0), (n_cty, n_day))
        Q = P_c_j #* case_count_all[e]
        env_data[e]["Q"] = Q

        print(f"for environment {e}: ")
        print(f"Background rate spans: {np.min(env_data[e]['mus']):.4f} to {np.max(env_data[e]['mus']):.4f}")
        print(f"Conditional intensity: {np.min(env_data[e]['lambda_t']):.4f} to {np.max(env_data[e]['lambda_t']):.4f}")
        print(f"Q- Average children: {np.min(env_data[e]['Q']):.4f} to {np.max(env_data[e]['Q']):.4f}")
        print(f"P_c(i,j) range: {np.min(env_data[e]['p_c_ij']):.4f} to {np.max(env_data[e]['p_c_ij']):.4f}")
        print(f"P_c(i,i) range: {np.min(env_data[e]['p_c_ii']):.4f} to {np.max(env_data[e]['p_c_ii']):.4f}")

    endog = [env["Q"].reshape(-1,1) for env in env_data]
    exog = [np.repeat(covariates_all[e], T, axis=0) for e in range(n_env)]
    event_freqs = [case_count_all[e].reshape(-1,1) for e in range(n_env)]
    # not using frequencies for now
    if args.verbose:
        print(f"Starting poisson model learning using Domain Generalization: {args.method} ")
    if args.method == "IRM":
        model = InvariantRiskMinimization(exog, endog, event_freqs, args)

    elif args.method == "ERM":
        model = EmpiricalRiskMinimizer(exog, endog, event_freqs, args)

    elif args.method == "MMD":
        model = MaximumMeanDiscrepancy(exog, endog, event_freqs, args)

    coef = model.solution()
    print(f"Estimated coeffecients across all environments in EM itr: {itr}:")
    print(coef)

    for e in range(n_env):
        # Update R0 with predictions for M-step
        env_data[e]["R0"] = model.predict(covariates_all[e])
        env_data[e]["R0"] = np.reshape(env_data[e]["R0"], (-1,1))

        # M-step
        mus = np.sum(env_data[e]["p_c_ii"], axis=1, keepdims=True)
        mus = mus.reshape(n_cty, n_day)
        mus = np.sum(mus, axis=1, keepdims=True) / n_day
        mus = np.clip(mus, 0, 1.5)
        env_data[e]["mus"] = mus

        obs = np.tril(np.arange(1, n_day+1)[:, None] - np.arange(1, n_day+1))
        weights = np.sum(env_data[e]["p_c_ij"].reshape(n_cty, n_day, n_day), axis=0)
        obs_1d = np.arange(1, n_day + 1)
        weights_1d = np.array([np.sum(weights[np.where(obs == time_diff)]) for time_diff in obs_1d])
        normalized_weights = np.divide(weights_1d, np.sum(weights_1d), where=np.sum(weights_1d) != 0)
        sample_size = 1000  # Choose an appropriate sample size
        sampled_times = np.random.choice(obs_1d, size=sample_size, p=normalized_weights)
        shape, loc, scale = weibull_min.fit(sampled_times, floc=0)
        env_data[e]["beta"] = np.clip(scale, 0.5, 20)
        env_data[e]["alpha"] = np.clip(shape, 0.5, 25)

        print(f"Environment {e+1}: Mus from {np.min(env_data[e]['mus']):.4f} to {np.max(env_data[e]['mus']):.4f}")
        print(f"Weibull scale: {env_data[e]['alpha']:.4f}, Shape: {env_data[e]['beta']:.4f}")

        if itr == 0:
            # Save the first iteration values
            mus_prev[e] = np.mean(env_data[e]["mus"], axis=0)
            theta_prev[e] = coef.flatten()
            alphas_prev[e] = env_data[e]["alpha"]
            betas_prev[e] = env_data[e]["beta"]
        else:
            # Calculate RMSR for convergence check
            mus_delta[e].append(np.sqrt(np.mean((mus_prev[e] - env_data[e]["mus"]) ** 2)))
            theta_delta[e].append(np.sqrt(np.mean((theta_prev[e] - coef) ** 2)))
            alpha_delta[e].append(np.sqrt((env_data[e]["alpha"] - alphas_prev[e]) ** 2))
            beta_delta[e].append(np.sqrt((env_data[e]["beta"] - betas_prev[e]) ** 2))

            # Save current values for next iteration
            mus_prev[e] = env_data[e]["mus"]
            theta_prev[e] = coef
            alphas_prev[e] = env_data[e]["alpha"]
            betas_prev[e] = env_data[e]["beta"]

    # Early stopping criteria
    if itr > 5:
        print("Commencing convergence check: ")
        converged = True
        for e in range(n_env):
            env_converged = (
                np.all(np.array(mus_delta[e][-5:]) < break_diff) and
                np.all(np.array(theta_delta[e][-5:]) < break_diff) and
                np.all(np.array(alpha_delta[e][-5:]) < break_diff) and
                np.all(np.array(beta_delta[e][-5:]) < break_diff)
            )
            if not env_converged:
                converged = False
                break

        if converged:
            print(f"Convergence criterion met at iteration {itr + 1}. Exiting EM loop.")

            # Save all plots to converged directory
            # Convergence plot
            plt.figure(figsize=(10, 6))
            plt.plot(np.arange(1, len(alpha_delta[0])+1), np.mean(alpha_delta, axis=0), label="Alpha Deltas", color="red")
            plt.plot(np.arange(1, len(beta_delta[0])+1), np.mean(beta_delta, axis=0), label="Beta Deltas", color="blue")
            plt.plot(np.arange(1, len(mus_delta[0])+1), np.mean(mus_delta, axis=0), label="Mus Deltas", color="green")
            plt.plot(np.arange(1, len(theta_delta[0])+1), np.mean(theta_delta, axis=0), label="0 Deltas", color="orange")
            plt.legend()
            plt.savefig(os.path.join(converged_dir, "convergence_plot.png"))
            plt.close()

            # Final comparison plots
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
            plt.tight_layout()
            plt.savefig(os.path.join(converged_dir, "final_comparison_plots.png"))
            plt.close()

            # Save model parameters
            avg_alpha = np.mean([env["alpha"] for env in env_data])
            avg_beta = np.mean([env["beta"] for env in env_data])
            avg_mus = np.mean([env["mus"] for env in env_data], axis=0)
            scaled_coef = coef.flatten() * cov_std

            model_params = {
                'method': args.method,
                'true_params': {
                    'alpha': args.true_alpha,
                    'beta': args.true_beta,
                    'mu': mu.flatten().tolist(),
                    'weights': true_weights.flatten().tolist()
                },
                'estimated_params': {
                    'alpha_per_env': [env["alpha"] for env in env_data],
                    'beta_per_env': [env["beta"] for env in env_data],
                    'mu_per_env': [env["mus"].flatten().tolist() for env in env_data],
                    'avg_alpha': avg_alpha,
                    'avg_beta': avg_beta,
                    'avg_mus': avg_mus.flatten().tolist(),
                    'scaled_coefficients': scaled_coef.tolist()
                },
                'convergence_metrics': {
                    'final_iteration': itr + 1,
                    'alpha_delta': np.mean(alpha_delta, axis=0).tolist(),
                    'beta_delta': np.mean(beta_delta, axis=0).tolist(),
                    'mus_delta': np.mean(mus_delta, axis=0).tolist(),
                    'theta_delta': np.mean(theta_delta, axis=0).tolist()
                },
                'mse_metrics': {
                    'alpha_mse': float((args.true_alpha - avg_alpha)**2),
                    'beta_mse': float((args.true_beta - avg_beta)**2),
                    'mus_mse': float(np.mean((mu.flatten() - avg_mus.flatten())**2)),
                    'weights_mse': float(np.mean((true_weights - scaled_coef)**2))
                }
            }

            # Save parameters to JSON file
            with open(os.path.join(converged_dir, 'model_parameters.json'), 'w') as f:
                json.dump(model_params, f, indent=4)

            break

    elapsed_time = time.time() - start_time
    print(f"Iteration {itr+1} completed in {elapsed_time:.2f} seconds.")
    print("----------------------------------------------------------")


print("EM Algorithm Completed.")

Final Model parameters

In [ ]:
avg_alpha = np.mean([env["alpha"] for env in env_data])
avg_beta = np.mean([env["beta"] for env in env_data])
avg_mus = np.mean([env["mus"] for env in env_data], axis=0)
scaled_coef = coef.flatten() * cov_std

Final Results compilation

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(np.arange(1, len(alpha_delta[0])+1), np.mean(alpha_delta, axis=0), label="Alpha Deltas", color="red")
plt.plot(np.arange(1, len(beta_delta[0])+1), np.mean(beta_delta, axis=0), label="Beta Deltas", color="blue")
plt.plot(np.arange(1, len(mus_delta[0])+1), np.mean(mus_delta, axis=0), label="Mus Deltas", color="green")
plt.plot(np.arange(1, len(theta_delta[0])+1), np.mean(theta_delta, axis=0), label="0 Deltas", color="orange")
plt.legend()
plt.savefig(os.path.join(checkpoint_dir, "final_convergence_plot.png"))
plt.close()

# Calculate averages across environments
avg_alpha = np.mean([env["alpha"] for env in env_data])
avg_beta = np.mean([env["beta"] for env in env_data])
avg_mus = np.mean([env["mus"] for env in env_data], axis=0)

# Create a figure with 3 subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Alpha and Beta comparison
params = ['Alpha', 'Beta']
true_vals = [args.true_alpha, args.true_beta]
est_vals = [avg_alpha, avg_beta]
x = np.arange(len(params))
width = 0.35

ax1.bar(x - width/2, true_vals, width, label='True Values', color='skyblue')
ax1.bar(x + width/2, est_vals, width, label='Estimated Values', color='lightcoral')
ax1.set_ylabel('Value')
ax1.set_title('True vs Estimated Alpha and Beta')
ax1.set_xticks(x)
ax1.set_xticklabels(params)
ax1.legend()

# Plot 2: True mu vs Estimated mu comparison across counties
x = np.arange(len(mu))
ax2.bar(x - width/2, mu.flatten(), width, label='True μ', color='skyblue')
ax2.bar(x + width/2, avg_mus.flatten(), width, label='Estimated μ', color='lightcoral')
ax2.set_xlabel('County Index')
ax2.set_ylabel('Background Rate (μ)')
ax2.set_title('True vs Estimated Background Rates')
ax2.set_xticks(x)
ax2.legend()

# Plot 3: True weights vs Estimated coefficients across dimensions
x = np.arange(len(true_weights))
scaled_coef = coef.flatten() * cov_std
ax3.bar(x - width/2, true_weights.flatten(), width, label='True Weights', alpha=0.7)
ax3.bar(x + width/2, scaled_coef, width, label='Estimated Coef', alpha=0.7)
ax3.set_xlabel('Coefficient Index')
ax3.set_ylabel('Value')
ax3.set_title('True vs Estimated Weights')
ax3.legend()

plt.tight_layout()
plt.savefig(os.path.join(checkpoint_dir, "final_comparison_plots.png"))
plt.close()

# Print MSE values
print(f"MSE for Alpha: {(args.true_alpha - avg_alpha)**2:.4f}")
print(f"MSE for Beta: {(args.true_beta - avg_beta)**2:.4f}")
print(f"MSE for Mus: {np.mean((mu.flatten() - avg_mus.flatten())**2):.4f}")
print(f"MSE for Weights: {np.mean((true_weights - scaled_coef)**2):.4f}")

print("\nFinal Model Parameters:")
print("-----------------------")
print(f"True Parameters:")
print(f"- Alpha: {args.true_alpha:.4f}")
print(f"- Beta: {args.true_beta:.4f}")
print(f"- Mu: {mu.flatten()}")
print(f"- Weights: {true_weights.flatten()}")
print("\nEstimated Parameters:")
print(f"- Alpha (avg across envs): {avg_alpha:.4f}")
print(f"- Beta (avg across envs): {avg_beta:.4f}")
print(f"- Mu (avg across envs): {avg_mus.flatten()}")
print(f"- Weights (scaled): {scaled_coef}")

# Print per-environment parameters
print("\nPer-Environment Parameters:")
for e in range(len(env_data)):
    print(f"\nEnvironment {e+1}:")
    print(f"- Alpha: {env_data[e]['alpha']:.4f}")
    print(f"- Beta: {env_data[e]['beta']:.4f}")
    print(f"- Mu range: [{np.min(env_data[e]['mus']):.4f}, {np.max(env_data[e]['mus']):.4f}]")